In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
#import dataset
games = pd.read_csv("../input/board-games-prediction-data/games.csv")

In [ ]:
games.describe()

In [ ]:
games.head(5)

In [ ]:
games.info()

In [ ]:
total = games.isnull().sum()
per_1 = (games.isnull().sum()/games.isnull().count())*100
per_2 = (round(per_1,1)).sort_values(ascending=False)
ts = pd.concat([total,per_2],axis=1, keys=['Total', '%'])
ts

In [ ]:
print(games.columns)
print(games.shape)

In [ ]:
plt.hist(games["average_rating"])
plt.show()

In [ ]:
#print the first row of all the games with zero scores
print(games[games["average_rating"]==0].iloc[0])


#print the first row of all the games with greater then zero scores
print(games[games["average_rating"]>0].iloc[0])

In [ ]:
#remove any rows without user review
games = games[games["average_rating"]>0]

#remove any rows with missing values
games = games.dropna(axis=0)


In [ ]:
plt.hist(games["average_rating"])
plt.show()

In [ ]:
total = games.isnull().sum()
per_1 = (games.isnull().sum()/games.isnull().count())*100
per_2 = (round(per_1,1)).sort_values(ascending=False)
ts = pd.concat([total,per_2],axis=1, keys=['Total', '%'])
ts

In [ ]:
#correlation matrix
corrmat = games.corr()
fig = plt.figure(figsize=(12,9))
sns.heatmap(corrmat,vmax=.8,square=True)

In [ ]:
#get all the columns
columns = games.columns.to_list()

#filter the column that we dont want
columns = [c for c in columns if c not in ["bayes_average_rating","average_rating","type","name","id"]]

#store the variable the we will be predicting
target = "average_rating"

In [ ]:
#generate training dataset
train = games.sample(frac=.8,random_state=1)

# select data that is not in train and put in test

test = games.loc[~games.index.isin(train.index)]

#print shape
print(train.shape)
print(test.shape)

In [ ]:
#import linear regression model

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error



In [ ]:
#intialize the model

LR = LinearRegression()

#fit model the training data
LR.fit(train[columns],train[target])


In [ ]:
#Generate the preddictions for the test set
predictions = LR.predict(test[columns])


In [ ]:
#compute error between our test predictions and actual values
mean_squared_error(predictions,test[target])

In [ ]:
#import the random forest model
from sklearn.ensemble import RandomForestRegressor

#intialize the model

RFR = RandomForestRegressor(n_estimators=100, min_samples_leaf=10,random_state=1)


In [ ]:
#fit the data
RFR.fit(train[columns],train[target])

In [ ]:
#make predictions 
predictions = RFR.predict(test[columns])

In [ ]:
#compute the error between our test predictions and the actual values
mean_squared_error(predictions,test[target])

In [ ]:
#make predication with the both models
rating_LR = LR.predict(test[columns].iloc[0].values.reshape(1,-1))
rating_RFR = RFR.predict(test[columns].iloc[0].values.reshape(1,-1))

In [ ]:
#print the predictions
print(rating_LR)
print(rating_RFR)

In [ ]:
test[target].iloc[0]